In [1]:
import pandas as pd

In [2]:
d = pd.read_csv('data/patches.csv', index_col=['id', 'name'])
print('Total of patches:', len(d))

d.head()

Total of patches: 5123


,,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
id,name,,,,,,
5299,plexi*dly*,74,102,55,NaN,NaN,NaN
5300,HB*Boston3,31,102,33,NaN,NaN,NaN
5301,_Clean****,24,31,28,NaN,NaN,NaN
5302,_Battles**,72,107,7,NaN,NaN,NaN
5303,_2tone****,72,57,96,NaN,NaN,NaN


## Remove patches from Zoom G3 v1

In [3]:
#d = d[d['plugin6'].notnull()]
#print('Total of patches after remove plugins from Zoom G3 v1 (only 3 effects):', len(d))

## Transform all presets in Zoom G3 v2

In [4]:
d = d.fillna(107)
d.head()

,,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
id,name,,,,,,
5299,plexi*dly*,74,102,55,107.0,107.0,107.0
5300,HB*Boston3,31,102,33,107.0,107.0,107.0
5301,_Clean****,24,31,28,107.0,107.0,107.0
5302,_Battles**,72,107,7,107.0,107.0,107.0
5303,_2tone****,72,57,96,107.0,107.0,107.0


## Move all 'None' to right

In [5]:
def swap_function(i=1):
    def swap(x):
        if x[f'plugin{i}'] == 107:
            x[f'plugin{i}'], x[f'plugin{i+1}'] = x[f'plugin{i+1}'], x[f'plugin{i}']

        return x

    return swap

for i in range(1, 7-1):
    for i in range(1, 7-1):
        d = d.apply(swap_function(i), axis=1)

In [6]:
[len(d[d[f'plugin{i}'] == 107]) for i in range(1, 7)]

[1514, 1558, 1649, 2197, 2862, 3487]

## Remove all presets with only one or two plugins

In [7]:
d = d[(d['plugin1'] != 107) & (d['plugin2'] != 107) & (d['plugin3'] != 107)]

print('Total of patches with more then two plugins:', len(d))
d.head()

Total of patches with more then two plugins: 3474


,,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
id,name,,,,,,
5299,plexi*dly*,74.0,102.0,55.0,107.0,107.0,107.0
5300,HB*Boston3,31.0,102.0,33.0,107.0,107.0,107.0
5301,_Clean****,24.0,31.0,28.0,107.0,107.0,107.0
5303,_2tone****,72.0,57.0,96.0,107.0,107.0,107.0
5304,Numb______,81.0,57.0,100.0,107.0,107.0,107.0


## Remove duplicates

In [8]:
d = d.drop_duplicates()
print('Total of patches after remove duplicated presets:', len(d))

d.head()

Total of patches after remove duplicated presets: 2162


,,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
id,name,,,,,,
5299,plexi*dly*,74.0,102.0,55.0,107.0,107.0,107.0
5300,HB*Boston3,31.0,102.0,33.0,107.0,107.0,107.0
5301,_Clean****,24.0,31.0,28.0,107.0,107.0,107.0
5303,_2tone****,72.0,57.0,96.0,107.0,107.0,107.0
5304,Numb______,81.0,57.0,100.0,107.0,107.0,107.0


## Remove patches with all plugins are equals

In [9]:
import operator
import functools

def columns_same_effect(effect):
    columns = ['plugin{}'.format(i) for i in range(1, 7)]
    #print([d[column] == str(effect) for column in columns])
    return functools.reduce(operator.and_, [d[column] == effect for column in columns])

# Remove effects with all effects equals
for i in range(117):
    d = d[~columns_same_effect(i)]

print(len(d), 'unique patches - disconsidering parameters')

2161 unique patches - disconsidering parameters


## Save

In [10]:
d.to_csv('data/patches-filtered.csv')